In [1]:
# import data file 

import numpy as np

data = np.genfromtxt('Health_dataset.txt',delimiter=',',dtype='<S4', skip_header=1);

line = open('Health_dataset.txt').readline()
header = line.strip('\n').split(',')
print(header)
print(data)

['smoker', 'alcoholic', 'drug_addict', 'exercise', 'lifespan']
[[b'yes1' b'yes2' b'no3' b'no' b'shor']
 [b'yes1' b'yes2' b'no3' b'yes' b'long']
 [b'no1' b'no2' b'yes3' b'no' b'shor']
 [b'no1' b'no2' b'no3' b'no' b'long']
 [b'no1' b'no2' b'no3' b'yes' b'long']
 [b'no1' b'yes2' b'yes3' b'no' b'shor']
 [b'yes1' b'yes2' b'yes3' b'yes' b'shor']
 [b'yes1' b'no2' b'yes3' b'no' b'shor']
 [b'no1' b'yes2' b'no3' b'yes' b'long']]


In [2]:
#### BASE ENTROPY OF Y
def Base_H(data):
    Unique, Counts = np.unique(data[:,-1], return_counts=True)
    print(Unique, Counts)
    global Base_Y
    Base_Y = 0
    for i in Counts:
        print(i)
        Base_Y += (i/sum(Counts))*np.log2(i/sum(Counts))
        print(-1*Base_Y)
    return;

Base_H(data);

[b'long' b'shor'] [4 5]
4
0.5199666673076944
5
0.9910760598382222


In [3]:
# CONDITIONAL ENTROPY OF Xs

def Condn_H(data):
    H_Ent = []

    for i in range(data.shape[1] - 1):
        Unique, Counts = np.unique(data[:, i], return_counts=True)
        Y_G_X = 0
        for ind in range(len(Unique)):
            Indexes = np.where(data[:,i] == Unique[ind])
            Unique_Target, Target_Counts = np.unique(data[Indexes, -1], return_counts=True)
            Y_X = 0
            for k in Target_Counts:
                Y_X += (k/sum(Target_Counts))*np.log2(k/sum(Target_Counts))
            Y_G_X += Counts[ind] * -Y_X / sum(Counts)
        H_Ent.append(Y_G_X)
    #print(H_Ent)
    return H_Ent;
Condn_H(data);


DECISION TREE LEARNING:
1. root = new node(Data)
2. train_tree(root)

def train_tree(node):
 - m = best attribute to split upon
 - let m be the test for node.
 - for each vaue of m, create decendent
 - partition data into decents by m
 - if the data is perfectly classified: STOP
 - Else: recurse with train_tree on each descentednt.
    
Splitting criteria:
1. misclification rate (minimize)
2. information gain (maximize) = ID3 algo - a greedy search algo using MI/IG 
3. ginni impurity ()

ID3 - searches for smallest tree consisant(perfectly classifies) with the data. 

  Write helper functions to calculate entropy and mutual information.
  • Write a function to train a stump (tree with only one level). Then call that function recursively to create the sub-trees.
• In the recursion, keep track of the depth of the current tree so you can stop growing the tree beyond the max-depth.
• Implement a function that takes a learned decision tree and data as inputs, and generates predicted labels. You can
write a separate function to calculate the error of the predicted labels with respect to the given (ground-truth) labels.
• Be sure to correctly handle the case where the specified maximum depth is greater than the total number of attributes.
• Be sure to handle the case where max-depth is zero (i.e. a majority vote classifier).

In [4]:
# DIVIDE STEP        #max depth = 3 or MI > 0

nested_dic = {'root':{}}
set_indc = []
header_list = []
global temp 
def make_tree(data_t, count, sub_nested_dic, colnames):
    H_Ent = Condn_H(data_t)
    print(data_t)
    print(H_Ent)
    choice_indc = np.argmin(H_Ent)
    header_list.append(colnames[choice_indc])
    print(colnames[choice_indc])
    print(choice_indc)
    Unique2, Counts2 = np.unique(data_t[:, choice_indc], return_counts=True)
    #print(Unique2, Counts2) # unique labels and their count in chosen column
    #nested_dic[header[choice_indc]] = {}
    sub_nested_dic.update({Unique2[i]:{} for i in range(len(Unique2))})
    print(sub_nested_dic)

    for ind2 in range(len(Unique2)): 
        # one cyle for each label category
        Indexes2 = np.where(data_t[:,choice_indc] == Unique2[ind2]) #indices in column for which each value of label exists.
        Unique_Target2, Target_Counts2 = np.unique(data_t[Indexes2, -1], return_counts=True)# divide Y value based on above X value.
        Y_X2 = 0
        count = count + 1
        print(Indexes2)
        # distribution of Y values for each label value of chosen X 
        #nested_dic[header[choice_indc]].append(Unique2[ind2])
        
        if count < 3:
            if len(Target_Counts2) != 1:
                
                #recalculate
                temp = np.copy(np.array(data_t[Indexes2]))
                temp = np.delete(temp, choice_indc, 1)
                print('Here', temp)
                del colnames[choice_indc]
                #del 
                
                #print(header[choice_indc], Unique2[ind2], )
                sub_nested_dic[Unique2[ind2]] = make_tree(temp, count, sub_nested_dic[Unique2[ind2]], colnames)
            else:
                sub_nested_dic[Unique2[ind2]] = Unique_Target2[0]
                    
        count = 0
    #print(sub_nested_dic)
    return sub_nested_dic; 

nested_dic['root'] = make_tree(data, 0, nested_dic['root'], header)
print(nested_dic)
print(header_list) 
print(nested_dic.items())


[[b'yes1' b'yes2' b'no3' b'no' b'shor']
 [b'yes1' b'yes2' b'no3' b'yes' b'long']
 [b'no1' b'no2' b'yes3' b'no' b'shor']
 [b'no1' b'no2' b'no3' b'no' b'long']
 [b'no1' b'no2' b'no3' b'yes' b'long']
 [b'no1' b'yes2' b'yes3' b'no' b'shor']
 [b'yes1' b'yes2' b'yes3' b'yes' b'shor']
 [b'yes1' b'no2' b'yes3' b'no' b'shor']
 [b'no1' b'yes2' b'no3' b'yes' b'long']]
[0.8999850522344306, 0.9838614413637048, 0.40107116382631236, 0.7616392191414825]
drug_addict
2
{b'no3': {}, b'yes3': {}}
(array([0, 1, 3, 4, 8]),)
Here [[b'yes1' b'yes2' b'no' b'shor']
 [b'yes1' b'yes2' b'yes' b'long']
 [b'no1' b'no2' b'no' b'long']
 [b'no1' b'no2' b'yes' b'long']
 [b'no1' b'yes2' b'yes' b'long']]
[[b'yes1' b'yes2' b'no' b'shor']
 [b'yes1' b'yes2' b'yes' b'long']
 [b'no1' b'no2' b'no' b'long']
 [b'no1' b'no2' b'yes' b'long']
 [b'no1' b'yes2' b'yes' b'long']]
[0.4, 0.5509775004326938, 0.4]
smoker
0
{b'no1': {}, b'yes1': {}}
(array([2, 3, 4]),)
(array([0, 1]),)
Here [[b'yes2' b'no' b'shor']
 [b'yes2' b'yes' b'long']]

Information Gain/Mutual Information: if we know X, how much does this reduce our uncertainty about Y. 

Create Test fucntion for classification:

- take tree and test file 
- read headers and the correspondong indices
- pass/traverse each row as input
- compare key with header and recurse over nesting of dictionary 
- when reached end key in nest, return value


In [20]:
# DECISION TREE TEST

data_test = np.genfromtxt('Health_datatest.txt',delimiter=',',dtype='<S4', skip_header=1);
header_list2 = header_list[:]
line = open('Health_datatest.txt').readline()
header2 = line.strip('\n').split(',')
print(header2)
#print(data_test)
print(header_list2)

def Tree_test(learned_tree, data_test2, header_l, ronum):
    
    r_n = np.size(data_test,0)
    for key, value in learned_tree.items():
        b = data_test2[header2.index(header_l[0])]
        if key == "root":
            Tree_test(value, data_test[ronum,:], header_l, ronum)
                
        elif (key == b):
            if type(value) is dict:
                del header_l[0]
                Tree_test(value, data_test[ronum,:], header_l, ronum)
            else:
                print(value)
                ronum = ronum+1
                header_l = []
                header_l = header_list[:]
                if ronum == r_n:
                    print(data_test2)
                    return
                else:
                    Tree_test(nested_dic, data_test[ronum,:], header_l, ronum)
        else:
            continue                    
        
        #check/compare key , if value is dictionary pass it to recurse again
        #if value is single , return value
    
Tree_test(nested_dic, data_test[0,:], header_list2, 0)

['smoker', 'alcoholic', 'drug_addict', 'exercise', 'lifespan']
['drug_addict', 'smoker', 'exercise']
b'shor'
b'long'
b'shor'
b'long'
b'long'
b'shor'
b'shor'
b'shor'
b'long'
[b'no1' b'yes2' b'no3' b'yes' b'long']


In [ ]:
# Fruit classifier based on color and size
Nested_Dic = {'Root':{"Red":{'Small':'Cherry','Big':'Apple'},'Blue':{'Small':'Blueberry'},'Green':{'Small':'Grapes','Big':'Green_Apple'}}}
print(Nested_Dic['Root'])

# Depth is number of splitting attributes. Here it is two - Color and Size
Depth = 0
Path = ''

def Recursive_Func(New_Dic, Depth, Path):
    if(type(New_Dic) == dict):
        Keys = New_Dic.keys()
        # Recurse for every key
        for key in Keys:
            # Add key to current path inside function call
            # Recurse one level deeper
            Recursive_Func(New_Dic[key], Depth + 1, Path + '\\' + key)
    else:
        # Add value as you have reached end of path and print path
        Path = Path + ' -> ' + New_Dic
        print(Path)
        # Return something to terminate this path
        return 0
    
# Call function
Recursive_Func(Nested_Dic, Depth, Path)